<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/amazoncustomerreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import dask.dataframe as dd
import gdown
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [23]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#file_id = '1TiwWTY4OT9UCAxOsZCTb9dCmGp34vnDX'
file_id = '1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK'
file_name = 'amazonreviews_train_small.csv'

# smalltoken_file_id = '1srwFqGjauA-00wJzenIJvvtd1vJldBmp'
# smalltoken_file_name = 'amazonreviews_train_small_tokenized_data.pt'

# Construct the download URL (with confirmation bypass)
url = f"https://drive.google.com/uc?export=download&id={file_id}"
# url_smalltoken = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
#gdown.download(url, "amazonreviews_train.csv", quiet=False)
gdown.download(url, file_name, quiet=False)

# gdown.download(url_smalltoken, smalltoken_file_name, quiet=False)



Downloading...
From (original): https://drive.google.com/uc?export=download&id=1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK
From (redirected): https://drive.google.com/uc?export=download&id=1DXgHh9TzHFEhXlElkJJD6gJA2B-vs8pK&confirm=t&uuid=3eddb359-9661-4083-888f-9f2061ea4fa2
To: /content/amazonreviews_train_small.csv
100%|██████████| 224M/224M [00:01<00:00, 199MB/s]


'amazonreviews_train_small.csv'

In [25]:
import pandas as pd

chunk_size = 100000  # Adjust chunk size as needed
chunks = []  # List to store chunk DataFrames

# Read the CSV in chunks
for chunk in pd.read_csv(file_name, chunksize=chunk_size):
    chunks.append(chunk)  # Store each chunk

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, axis=0)

# Display the first few rows
print(df.head())


   polarity                                              title  \
0         2                     Stuning even for the non-gamer   
1         2              The best soundtrack ever to anything.   
2         2                                           Amazing!   
3         2                               Excellent Soundtrack   
4         2  Remember, Pull Your Jaw Off The Floor After He...   

                                                text  
0  This sound track was beautiful! It paints the ...  
1  I'm reading a lot of reviews saying that this ...  
2  This soundtrack is my favorite music of all ti...  
3  I truly like this soundtrack and I enjoy video...  
4  If you've played the game, you know how divine...  


In [26]:
df["polarity"].value_counts()

,count
polarity,
2,252782
1,247217


In [27]:
print(df.shape)

(499999, 3)


In [28]:
df.describe()

,polarity
count,499999.000000
mean,1.505565
std,0.499970
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


In [29]:
df.dtypes

,0
polarity,int64
title,object
text,object


In [30]:

# Basic text cleaning- Remove special characters or puncatuation marks
df['cleaned_reviews'] = df['text'].str.replace('[^a-zA-Z\s]', '', regex=True)

In [31]:
df.head(5)

,polarity,title,text,cleaned_reviews
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...,This sound track was beautiful It paints the s...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,Im reading a lot of reviews saying that this i...
2,2,Amazing!,This soundtrack is my favorite music of all ti...,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine...",If youve played the game you know how divine t...


In [32]:
from transformers import BertTokenizer


# *   What is the the BERT Tokenizer doing?
# *   It converts the cleaned reviews to list because hugging face tokenizers work with lists
# * BERT appies word piece tokenization . Converts text into subword tokens and assigns each token a unique id
# * padding = True . Ensures that all tokenized sequences in the batch are the same length by adding padding tokens to shorter sequences. This is important because transformers process fixed-length inputs in batches
# * truncation = True . Shortens any long reviews that exceed the model's maximum token lime ( e:g 512 for BERT). Ensures that excesskive long reviews dont break the models input size limit
# * converts the tokenzied output into a pytorch tensor (torch.tensor) format
# * required for training or inference in pytorch-based models
# * if using tensorflow , we have to pass th e parameter as return_tensors = "tf"

# # Initialize a tokenizer from a pretrained model (e.g., BERT)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # Tokenize the reviews
# tokens = tokenizer(df['cleaned_reviews'].tolist(), padding=True, truncation=True, return_tensors="pt")




In [33]:
# import torch

# # Save the tokenized output (dictionary of tensors)
# torch.save(tokens, 'amazonreviews_train_small_tokenized_data.pt')



*   Do not remove stop words from the tokens if we are using the dataset to do
sentiment analysis (classifying reviews as positive / negative or neutral)

*  Do not remove stop words from the tokens if we are classifying reviews into various categories (electronics, clothing etc) and may be summarization of the review



In [34]:
# # Display tokenized output (first 2 examples)
# print(tokens.input_ids[:2])
# print(tokens.attention_mask[:2])

In [35]:
# import torch
# from transformers import BatchEncoding

# # Add BatchEncoding to the safe globals list
# torch.serialization.add_safe_globals([BatchEncoding])

# tokens = torch.load(smalltoken_file_name, map_location=torch.device("cpu"),weights_only=False)
# # print(tokens)  # Check the content


# input_ids = tokens['input_ids']
# attention_mask = tokens['attention_mask']

# print(input_ids.shape)         # Should show the shape of input_ids
# print(attention_mask.shape)    # Should show the shape of attention_mask


In [36]:

# # Extract input IDs and attention mask
# input_ids = tokens['input_ids']
# attention_mask = tokens['attention_mask']
# print(input_ids)
# print(attention_mask)

In [37]:
# # Set model to evaluation mode (important for inference)
# from transformers import BertModel
# import torch

# model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()

# # Get the output of the model
# with torch.no_grad():
#     outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# # Extract the hidden states from the last layer
# last_hidden_states = outputs.last_hidden_state

# print(last_hidden_states.shape)  # Shape: [batch_size, seq_len, hidden_size]
